Import data

In [1]:
import pandas as pd
import os

# Path to data
data_path = '../data/Data/'

Load data

In [2]:
# Create an empty list to store DataFrames
dataframes = []
index = 0

# Traverse through user directories
for user_folder in os.listdir(data_path):
	user_folder_path = os.path.join(data_path, user_folder)

	# Check if it's a directory
	if os.path.isdir(user_folder_path):
		trajectory_dir = os.path.join(user_folder_path, 'Trajectory')
		
		# Check if 'Trajectory' directory exists
		if os.path.exists(trajectory_dir) and os.path.isdir(trajectory_dir):
			# Loop through trajectory files for each user
			print(trajectory_dir)
			for trajectory_file in os.listdir(trajectory_dir):
				if trajectory_file.endswith('.plt'):
					trajectory_file_path = os.path.join(trajectory_dir, trajectory_file)
					
					df = pd.read_csv(trajectory_file_path, header=None, skiprows=6)
					df.insert(0, 'person', index)
					dataframes.append(df)
					index += 1

./data/Data/000\Trajectory
./data/Data/001\Trajectory
./data/Data/002\Trajectory
./data/Data/003\Trajectory
./data/Data/004\Trajectory
./data/Data/005\Trajectory
./data/Data/006\Trajectory
./data/Data/007\Trajectory
./data/Data/008\Trajectory
./data/Data/009\Trajectory
./data/Data/010\Trajectory
./data/Data/011\Trajectory
./data/Data/012\Trajectory
./data/Data/013\Trajectory
./data/Data/014\Trajectory
./data/Data/015\Trajectory
./data/Data/016\Trajectory
./data/Data/017\Trajectory
./data/Data/018\Trajectory
./data/Data/019\Trajectory
./data/Data/020\Trajectory
./data/Data/021\Trajectory
./data/Data/022\Trajectory
./data/Data/023\Trajectory
./data/Data/024\Trajectory
./data/Data/025\Trajectory
./data/Data/026\Trajectory
./data/Data/027\Trajectory
./data/Data/028\Trajectory
./data/Data/029\Trajectory
./data/Data/030\Trajectory
./data/Data/031\Trajectory
./data/Data/032\Trajectory
./data/Data/033\Trajectory
./data/Data/034\Trajectory
./data/Data/035\Trajectory
./data/Data/036\Trajectory
.

Concat data

In [3]:
# Concatenate all DataFrames into one
gps_data = pd.concat(dataframes, ignore_index=True)
gps_data

,person,0,1,2,3,4,5,6
0,0,39.984702,116.318417,0,492.000000,39744.120185,2008-10-23,02:53:04
1,0,39.984683,116.318450,0,492.000000,39744.120255,2008-10-23,02:53:10
2,0,39.984686,116.318417,0,492.000000,39744.120313,2008-10-23,02:53:15
3,0,39.984688,116.318385,0,492.000000,39744.120370,2008-10-23,02:53:20
4,0,39.984655,116.318263,0,492.000000,39744.120428,2008-10-23,02:53:25
...,...,...,...,...,...,...,...,...
24876973,18669,40.914867,111.710500,0,3802.493438,39521.152731,2008-03-14,03:39:56
24876974,18669,40.914267,111.710333,0,3795.931759,39521.153669,2008-03-14,03:41:17
24876975,18669,40.912467,111.710667,0,3795.931759,39521.154884,2008-03-14,03:43:02
24876976,18669,40.911517,111.711317,0,3779.527559,39521.155185,2008-03-14,03:43:28


Add labels

In [4]:
# Assign column names
column_names = ['Person ID', 'Latitude', 'Longitude', '0', 'Altitude', 'NumDays', 'Date', 'Time']

# Rename columns in the DataFrame
gps_data.columns = column_names
gps_data

,Person ID,Latitude,Longitude,0,Altitude,NumDays,Date,Time
0,0,39.984702,116.318417,0,492.000000,39744.120185,2008-10-23,02:53:04
1,0,39.984683,116.318450,0,492.000000,39744.120255,2008-10-23,02:53:10
2,0,39.984686,116.318417,0,492.000000,39744.120313,2008-10-23,02:53:15
3,0,39.984688,116.318385,0,492.000000,39744.120370,2008-10-23,02:53:20
4,0,39.984655,116.318263,0,492.000000,39744.120428,2008-10-23,02:53:25
...,...,...,...,...,...,...,...,...
24876973,18669,40.914867,111.710500,0,3802.493438,39521.152731,2008-03-14,03:39:56
24876974,18669,40.914267,111.710333,0,3795.931759,39521.153669,2008-03-14,03:41:17
24876975,18669,40.912467,111.710667,0,3795.931759,39521.154884,2008-03-14,03:43:02
24876976,18669,40.911517,111.711317,0,3779.527559,39521.155185,2008-03-14,03:43:28


Recude data to Beijing Metro area only

In [6]:
gps_data = gps_data[gps_data['Latitude'] >= 39.43]
gps_data = gps_data[gps_data['Latitude'] <= 40.63]
gps_data = gps_data[gps_data['Longitude'] >= 115.8]
gps_data = gps_data[gps_data['Longitude'] <= 117.2]

gps_data

,Person ID,Latitude,Longitude,0,Altitude,NumDays,Date,Time
0,0,39.984702,116.318417,0,492.000000,39744.120185,2008-10-23,02:53:04
1,0,39.984683,116.318450,0,492.000000,39744.120255,2008-10-23,02:53:10
2,0,39.984686,116.318417,0,492.000000,39744.120313,2008-10-23,02:53:15
3,0,39.984688,116.318385,0,492.000000,39744.120370,2008-10-23,02:53:20
4,0,39.984655,116.318263,0,492.000000,39744.120428,2008-10-23,02:53:25
...,...,...,...,...,...,...,...,...
24876951,18668,39.988783,116.299000,0,114.829396,39495.446632,2008-02-17,10:43:09
24876952,18668,39.989500,116.298667,0,98.425197,39495.447176,2008-02-17,10:43:56
24876953,18668,39.990067,116.298350,0,131.233596,39495.447523,2008-02-17,10:44:26
24876954,18668,39.989517,116.298417,0,150.918635,39495.448183,2008-02-17,10:45:23


Save data

In [5]:
# Save DataFrame to a CSV file
gps_data.to_csv('../model/gps_data.csv', index=False)

Load Labels

In [ ]:
# Create an empty list to store DataFrames
dataframes = []

# Traverse through user directories
for user_folder in os.listdir(data_path):
	user_folder_path = os.path.join(data_path, user_folder)

	labels_file_path = os.path.join(user_folder_path, 'labels.txt')
					
	# Check if the labels file exists
	if os.path.exists(labels_file_path):
		labels_df = pd.read_csv(labels_file_path, sep='\t')
		
		dataframes.append(labels_df)

labels_data = pd.concat(dataframes, ignore_index=True)
labels_data

In [ ]:
# Save DataFrame to a CSV file
labels_data.to_csv('../model/labels_data.csv', index=False)